# Lesson 10.3: Other Advanced Prompting Strategies

---

In Lesson 10.2, we explored **Chain-of-Thought (CoT) Prompting**, a powerful technique that helps LLMs reason step-by-step. However, the field of Prompt Engineering is constantly evolving, and there are many other advanced strategies that can significantly improve LLM performance on complex tasks. This lesson will introduce strategies such as **Tree-of-Thought (ToT)**, **Self-Consistency**, **Retrieval-Augmented Prompting**, and **Prompt Chaining**, along with a practical exercise to compare their effectiveness.

## 1. Tree-of-Thought (ToT): Exploring Multiple Thought Paths

**Tree-of-Thought (ToT)** is a Prompting strategy that extends CoT, allowing the LLM to explore and evaluate multiple possible thought paths, rather than just a single linear chain.

* **Concept:** Instead of generating a single thought chain, ToT encourages the LLM to generate a "tree" of thoughts. Each "node" in the tree is an intermediate thought state, and the LLM can branch out to explore different lines of reasoning.
* **Process:**
    1.  **Problem Decomposition:** The LLM breaks down the problem into smaller steps.
    2.  **Generate Multiple Options:** At each step, the LLM generates multiple possible next thoughts or actions.
    3.  **Evaluation:** The LLM (or another model/evaluation function) evaluates the "quality" or "promise" of each thought path.
    4.  **Search:** Uses search algorithms (e.g., Breadth-First Search - BFS, Depth-First Search - DFS, or Beam Search) to explore the thought tree and select the best path leading to a solution.
* **Pros:**
    * Significantly improves the ability to solve complex problems, especially those with multiple steps and requiring exploration of the solution space.
    * Reduces the risk of getting stuck in a flawed thought chain.
* **Cons:**
    * More complex to implement and manage.
    * More computationally expensive and costly (more LLM calls).
* **Applications:** Solving advanced mathematical problems, complex planning, multi-step logical reasoning.

![A tree diagram with multiple branches, each representing a thought path](https://placehold.co/600x400/aaddcc/ffffff?text=Tree-of-Thought+Diagram)


---

## 2. Self-Consistency: Generating Multiple Answers and Choosing the Most Popular

**Self-Consistency** is a simple yet effective technique to improve the reliability of LLM responses by leveraging their stochastic nature.

* **Concept:** Instead of just asking the LLM to generate a single answer, you ask it to generate **multiple independent answers** to the same question (typically with `temperature > 0`). Then, you aggregate these answers and choose the **most popular (majority vote)** or the **best** answer (e.g., by another LLM).
* **Process:**
    1.  **Generate Multiple Thought Chains/Answers:** Run the same prompt (often a CoT prompt) multiple times to have the LLM generate different thought chains and answers.
    2.  **Extract Final Answer:** From each thought chain, extract the final answer.
    3.  **Aggregate/Select:** Use a voting mechanism (e.g., counting the frequency of answers) or a judge LLM to select the final answer.
* **Pros:**
    * Improves accuracy and reliability, especially for problems with a single correct answer.
    * Simpler to implement than ToT.
* **Cons:**
    * Increases computational cost and API cost (more LLM calls).
* **Applications:** Mathematical problems, logical puzzles, high-accuracy Q&A.

![Multiple LLM responses converging to a single, consistent answer](https://placehold.co/600x400/ccddeeff/ffffff?text=Self-Consistency+Flow)


---

## 3. Retrieval-Augmented Prompting: Combining RAG with Advanced Prompting Techniques

We've learned about **Retrieval-Augmented Generation (RAG)** as a way to provide external context to LLMs. **Retrieval-Augmented Prompting** is the act of combining RAG with advanced Prompting techniques (like CoT, ToT) to optimize the use of retrieved information.

* **Concept:** Instead of simply inserting retrieved context into the prompt, you use Prompting techniques to guide the LLM on how to interact with that context more effectively.
* **Techniques:**
    * **CoT with Context:** Ask the LLM to think step-by-step about how to use the context to answer the question. Example: "Based on the following context, think step-by-step to answer the question. Context: [Text]. Question: [Question]"
    * **Context Analysis:** Ask the LLM to first summarize or extract key points from the context before answering the question.
    * **Factual Consistency Check:** After the LLM generates a response, ask it to cross-reference that response with the original context to ensure factual consistency.
* **Pros:**
    * Reduces hallucinations by forcing the LLM to stick to the provided information.
    * Improves accuracy and relevance of responses.
    * Enhances explainability by showing how the LLM used the context.
* **Cons:**
    * Can increase prompt complexity and the number of LLM calls.
* **Applications:** Q&A systems, knowledge chatbots, document summarization.

![A RAG pipeline with CoT steps integrated](https://placehold.co/600x400/ddeeff/ffffff?text=RAG+with+CoT)


---

## 4. Prompt Chaining: Building Complex Prompt Sequences

**Prompt Chaining** (also known as Chain of Prompts) is an organizational technique where a large task is broken down into multiple smaller tasks, and each smaller task is handled by a separate prompt. The output of one prompt becomes the input for the next.

* **Concept:** Instead of trying to solve the entire problem in a single prompt, you create a sequence of prompts, each performing a specific logical step.
* **Process:**
    1.  **Task Decomposition:** Break down a complex task into sequential steps.
    2.  **Design Prompt for Each Step:** Create a separate prompt for each step, instructing the LLM to perform that specific task.
    3.  **Connect Output/Input:** The LLM's output from one prompt is passed as input to the next prompt.
* **Pros:**
    * **Improved Accuracy:** The LLM can focus on a smaller task at a time, reducing the chance of errors.
    * **Debuggability:** Easier to pinpoint which step caused an error.
    * **Modularity:** Individual prompts can be reused in other chains.
    * **Flow Control:** Allows for tighter control over the processing flow.
* **Cons:**
    * Increased latency (multiple sequential LLM calls).
    * Increased cost (each LLM call costs money).
* **Applications:** Multi-step text processing (e.g., extraction -> classification -> summarization), Agents (each step in a LangGraph graph can be a prompt chain).

![A sequence of prompts, each feeding into the next](https://placehold.co/600x400/aaccaa/ffffff?text=Prompt+Chaining+Flow)


---

## 5. Practical: Comparing the Effectiveness of Different Prompting Strategies on the Same Problem

We will use a problem that requires logical/mathematical reasoning and compare the effectiveness of:
1.  Basic Prompt (no CoT)
2.  Zero-shot CoT
3.  Self-Consistency (simple)
4.  Prompt Chaining (to decompose the problem)

**Preparation:**
* Ensure you have the `langchain-openai` library installed.
* Set the `OPENAI_API_KEY` environment variable.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai

import os
from typing import List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from collections import Counter # To count frequencies for Self-Consistency
import json # For JSON parsing in Prompt Chaining

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7) # Use temperature > 0 for Self-Consistency

# Example problem
problem = "If today is Tuesday, what day of the week will it be in 75 days?"

# --- 1. Basic Prompt (Baseline) ---
print("--- Strategy: Basic Prompt ---")
basic_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an intelligent assistant. Answer the following question."),
    ("user", problem)
])
basic_chain = basic_prompt | llm | StrOutputParser()
response_basic = basic_chain.invoke({})
print(f"Basic Response: {response_basic}\n")

# --- 2. Zero-shot CoT ---
print("--- Strategy: Zero-shot CoT ---")
zero_shot_cot_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an intelligent assistant. Think step by step before providing the final answer."),
    ("user", problem)
])
zero_shot_cot_chain = zero_shot_cot_prompt | llm | StrOutputParser()
response_zero_shot_cot = zero_shot_cot_chain.invoke({})
print(f"Zero-shot CoT Response: {response_zero_shot_cot}\n")

# --- 3. Self-Consistency (Simple) ---
# Run Zero-shot CoT multiple times and pick the most frequent final answer
print("--- Strategy: Self-Consistency ---")
num_samples = 5 # Number of runs to generate different responses
final_answers = []

for i in range(num_samples):
    print(f"  Running Self-Consistency iteration {i+1}...")
    # Get CoT response and try to extract the final answer
    cot_response = zero_shot_cot_chain.invoke({})
    # Assume the final answer is after "Answer:" or the last non-empty line
    answer_lines = cot_response.split('\n')
    extracted_answer = ""
    for line in reversed(answer_lines):
        if "Answer:" in line:
            extracted_answer = line.split("Answer:", 1)[1].strip()
            break
        elif line.strip(): # If no "Answer:", take the last non-empty line
            extracted_answer = line.strip()
            break
    
    if extracted_answer:
        final_answers.append(extracted_answer)
    print(f"    Extracted response: {extracted_answer}")

if final_answers:
    most_common_answer = Counter(final_answers).most_common(1)[0][0]
    print(f"Self-Consistency Response (most common from {num_samples} samples): {most_common_answer}\n")
else:
    print("No answers extracted for Self-Consistency.\n")

# --- 4. Prompt Chaining ---
print("--- Strategy: Prompt Chaining ---")

# Step 1: Extract necessary information
extract_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a question analysis assistant. From the following question, extract: 1. The current day mentioned. 2. The number of days in the future. Respond in JSON."),
    ("user", problem)
])
extract_chain = extract_prompt | llm | StrOutputParser()
extracted_info_raw = extract_chain.invoke({})

try:
    extracted_info = json.loads(extracted_info_raw)
    current_day = extracted_info.get("The current day mentioned") # Adjust key to match LLM output if different
    days_ahead = extracted_info.get("The number of days in the future") # Adjust key to match LLM output if different
    print(f"  Step 1 (Extraction): Current Day: {current_day}, Days Ahead: {days_ahead}")

    # Step 2: Calculate the day of the week
    if current_day and days_ahead is not None:
        calculate_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a day calculation assistant. If today is {current_day}, what day of the week will it be in {days_ahead} days? Only respond with the day name."),
            ("user", "Calculate and provide the result.")
        ])
        calculate_chain = calculate_prompt | llm | StrOutputParser()
        final_response_chained = calculate_chain.invoke({"current_day": current_day, "days_ahead": days_ahead})
        print(f"  Step 2 (Calculation): {final_response_chained}")
        print(f"Prompt Chaining Response: {final_response_chained}\n")
    else:
        print(f"  Error extracting information: {extracted_info_raw}")
        print("Prompt Chaining Response: Could not complete due to extraction error.\n")

except json.JSONDecodeError:
    print(f"  Error parsing JSON from extraction step: {extracted_info_raw}")
    print("Prompt Chaining Response: Could not complete due to format error.\n")
except Exception as e:
    print(f"  Error during Prompt Chaining: {e}")
    print("Prompt Chaining Response: An error occurred in the chain.\n")

# --- 5. (Conceptual) Tree-of-Thought (ToT) ---
print("--- Strategy: Tree-of-Thought (ToT) (Conceptual) ---")
print("ToT is more complex to implement directly in a simple example.")
print("It would involve the LLM generating multiple 'thoughts' or 'actions' at each step,")
print("then an evaluation function scoring these thoughts,")
print("and a search algorithm traversing the tree to find the best path.")
print("Example: The LLM might try different calculations or different ways to analyze the problem,")
print("then self-evaluate or be evaluated by another LLM to choose the most correct path.\n")

# --- 6. (Conceptual) Retrieval-Augmented Prompting ---
print("--- Strategy: Retrieval-Augmented Prompting (Conceptual) ---")
print("This strategy combines RAG with advanced Prompting techniques.")
print("Example: To solve the problem above, if we had a database of 'Calendars' or 'How to calculate days of the week,'")
print("we would:")
print("1. Query the database to retrieve relevant information (e.g., 'number of days in a week').")
print("2. Inject this information into the prompt along with the question and ask the LLM to think step-by-step (CoT) based on the retrieved information.")
print("This helps the LLM have more reliable data for reasoning, reducing 'hallucinations'.\n")

print("--- End of Practical ---")